# Daily Add New Watched Users
Iterate all non-watched users (no `depth` assigned) and check if their number of appearances is greater than the threshold (`min_appearances_before_watched`) until one of the following happens:
* maximum of total watched users (`max_watched_users`) is reached 
* a daily increase of watched users (`max_daily_increase`) is reached 
* a percentual daily increase of watched users (`max_daily_increase_ratio`) is reached

In [ ]:
# required imports to access api_db, misc, misc.CONFIG, ...
import sys
sys.path = ['.', '..', '../..'] + sys.path
from collection import *

### Conditional Execution
Each file needs to verify if it should be executed or not based on the configurations (for some files this is not optional but all should have this section, even if it is tautological). Example:
```python
if not misc.CONFIG["collection"]["execute_this_script"]: exit()
```

In [ ]:
# Conditional execution
pass

<hr>
<h1 align="center">driver code</h1>

In [ ]:
from pytictoc import TicToc

In [ ]:
with TicToc():
    print("Loading seed_ids from database...", end="", flush=True)
    seed_ids = [s["_id"] for s in api_db.col_users.find({"depth": 0}, {}).limit(len(misc.CONFIG["seed"]["usernames"]))]
    print("got %d seed users, done." % len(seed_ids))

In [ ]:
conf_lim = misc.CONFIG["collection"]["limits"]
current_users = api_db.col_users.count_documents({"depth": {"$lte": 2}})
todays_limit = int(max(0, min(
    conf_lim["max_daily_increase"],
    current_users * conf_lim["max_daily_increase_ratio"],
    conf_lim["max_watched_users"] - current_users
)))

In [ ]:
print("Today's limit for new users is %d." % todays_limit)

In [ ]:
import re

In [ ]:
find_params = {
    "depth": {"$exists": False},
    "unwatched": {"$exists": False}, # set to mark accounts that should not be watched
    "appearances": {"$gte": conf_lim["min_appearances_before_watched"]},
    "location": {"$not": re.compile(r'brasil', re.IGNORECASE)}
}

with TicToc():
    print("Processing new users (silent db-only)...", end="", flush=True)
    # get the user ids
    user_ids = [u["_id"] for u in api_db.col_users.find(find_params, {}, no_cursor_timeout=True).limit(todays_limit)]
    # upsert them with the appropriate depth
    insert_user_ids(user_ids, depth=2)
    print("started watching %d new users, done." % len(user_ids))

In [ ]:
print("DONE")